## Run the feature_extraction.py first!!!!!!!!!!!


In [ ]:
import pandas as pd
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm
from textblob import TextBlob
from scipy.sparse import coo_matrix, hstack
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')




In [ ]:
train=pd.read_csv("./data/X_train.csv")
submission = pd.read_csv("./data/X_test.csv")

## Stemn the text and summary in X_train.csv

In [ ]:
dftrain = pd.DataFrame({'Summary': train['Summary'], 'Text': train['Text']})
stmer=SnowballStemmer("english",ignore_stopwords=True)

def stem(text):
    return ' '.join([stmer.stem(word) for word in word_tokenize(text)])
#df['test']=np.nan
#tqdm.pandas(desc="my bar!")
dftrain['Summary']=dftrain['Summary'].fillna("").apply(stem)
dftrain['Test']=dftrain['Text'].fillna("").apply(stem)

In [ ]:
dftrain.to_csv("./data/dftrain_c.csv", index=False)

## Stemn the text and summary in X_test.csv

In [ ]:
dfsub = pd.DataFrame({'Summary': submission['Summary'], 'Text': submission['Text']})
stmer=SnowballStemmer("english",ignore_stopwords=True)

def stem(text):
    return ' '.join([stmer.stem(word) for word in word_tokenize(text)])
#df['test']=np.nan
#tqdm.pandas(desc="my bar!")
dfsub['Summary']=dfsub['Summary'].fillna("").apply(stem)
dfsub['Test']=dfsub['Text'].fillna("").apply(stem)

In [ ]:
dfsub.to_csv("./data/dfsub_c.csv", index=False)

In [ ]:
df=pd.read_csv('./data/dfsub_c.csv')


## Combine stemmer together

In [ ]:
dftrain['Total'] = dftrain['Summary'] + dftrain['Test']

dfsub['Total'] = dfsub['Summary'] + " " + dfsub['Test']


TFID the stemmer

In [ ]:
tv = TfidfVectorizer(max_df=0.999, min_df=0.005, max_features=10000)
train_tfid = tv.fit_transform(dftrain['Total'].apply(lambda x: np.str_(x)))
#train_tfdf = pd.DataFrame(train_tfid.toarray(), columns=tv.get_feature_names())
train_tfdf = pd.DataFrame(train_tfid.toarray(), columns=tv.get_feature_names_out())
train_tfdf

In [ ]:
Sub_tfid = tv.transform(dfsub['Total'].apply(lambda x: np.str_(x)))
#Sub_tfdf = pd.DataFrame(Sub_tfid.toarray(), columns=tv.get_feature_names())
Sub_tfdf = pd.DataFrame(Sub_tfid.toarray(), columns=tv.get_feature_names_out())
Sub_tfdf

Concate together for further prediction

## Sentiment analisys of text and summary

In [ ]:
def pop(s):
    a,b=TextBlob(s).sentiment
    return a

train['Pop_text']=train['Text'].fillna("").apply(pop)

train['Pop_Summary']=train['Summary'].fillna("").apply(pop)

In [ ]:


submission['Pop_text']=submission['Text'].fillna("").apply(pop)

submission['Pop_Summary']=submission['Summary'].fillna("").apply(pop)

## Concate together for further prediction

In [ ]:
train_num=train.drop(columns=['Id', 'ProductId', 'UserId', 'Text', 'Summary','Score'])
train_final=pd.concat([train_num,train_tfdf],axis=1)

In [ ]:
sub_num=submission.drop(columns=['Id', 'ProductId', 'UserId', 'Text', 'Summary','Score'])
sub_final=pd.concat([sub_num,Sub_tfdf],axis=1)

## Split for train and test set and build model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split( train_final, train['Score'],test_size=0.25,random_state=0)

In [ ]:
model = LinearRegression(n_jobs=-1).fit(x_train, y_train)
Y_predictions = model.predict(x_test)
submission['Score'] = model.predict(sub_final)
def to_score(score):
    if score < 1:
        return 1
    elif score > 5:
        return 5
    else:
        return round(score)

submission['Score'] = submission['Score'].apply(to_score)
submission_file = submission[['Id', 'Score']]
submission_file.to_csv("./data/submission.csv", index=False)

In [ ]:
from sklearn.metrics import  mean_squared_error
from math import sqrt
print("RMSE on testing set = ", sqrt(mean_squared_error(y_test , Y_predictions)))

In [ ]:
#plot the linear regression graph
plt.scatter(y_test, Y_predictions)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.show()


